<img src="marine_region.jpg"/>
<h1> Marine Regions RESTful API Walkthrough</h1>
<p> MarineRegion's REST API can be accessed using python request library. All API calls to marineregions can return either a JSON file or a XML depending on the extension specified in the call. For our purposes we'll be using the <b>json</b> extension exclusively as it provides a better return for python to parse.</p>
<p>the Recommended version of python is <b>Python 3.7+</b> which can be downloaded from <a href="https://python.org/">this</a> link.<br>The required python libraries are listed in the accompanying environment.yml file</p>
<hr>
<h2> Python Imports </h2>
<p>Always run the cell below to initialize your python running environment. The requests package will be used to make calls to the OBIS API return JSON objects<br>
    More information about requests can be obtained <a href="https://requests.readthedocs.io/">here</a></p>

In [ ]:
# Import requests and set the Marine Regions API base URL. 
import requests
import json
try:
    import folium
except ModuleNotFoundError:
    print('Folium is not installed. Maps will not work.')
    
# Convenience function to pretty print json objects and python dicts
def print_json(myjson):
    print(json.dumps(
        myjson,
        sort_keys=True,
        indent=4,
        separators=(',', ': ')
    ))
# Initialize the base URL for Marine Regions. This variable is unchanged for every api call 
MARINE_REGIONS_URL = "http://marineregions.org/rest"
API_EXT = "json" # sets the return format for every MarineRegion API query

## Get Gazetteer Records by Name

### By Name, with 'like' (defaults to True if not included)
When the argument for '**like**' (bool) is not provided, the gazetteer API will match everything **before** and **after** your search term.

The argument '**fuzzy**' (bool) will use the Levenshtein algorithm to match nearest records. By default this is set as '**false**'

    Note: Boolean values should be set using 'true' or 'false' python strings.

In [ ]:
# Lets try query using the country Belgian as the name value
name = 'Belgian'

# /getGazetteerRecordsByName.json
req = requests.get(f'{MARINE_REGIONS_URL}/getGazetteerRecordsByName.{API_EXT}/{name}/')
print_json(req.json())

### By Name, without 'like'
When setting like to false make sure you provide the exact name of Gazetteer resource as it appears in MarineRegions

In [ ]:
# Setting the 
name = "Belgian 24 NM"
like = 'false' #Defaults to true

# getGazetteerRecordsByName.json/
req = requests.get(f'{marineRegions_url}/getGazetteerRecordsByName.{API_EXT}/{name}/{like}/')
req.json()

## Get Multiple Gazetteers By Name

Using the Call getGazetteerRecordsByNames.json, first provide the values for like and fuzzy before providing the multiple name values.

In [ ]:
# Lets keep the defaults for like and fuzzy
like = 'true' 
fuzzy = 'false'

# build a slash separated list of names to query by
names = ["Canadian Coast", "Mexican Coast", "Norwegian Coast"]
names_string = '/'.join(names)

query_string = f'{marineRegions_url}/getGazetteerRecordsByNames.{API_EXT}/{like}/{fuzzy}/{names_string}/'
req = requests.get(query_string)
print_json(req.json())

## Getting Gazetteers by Sources

Querying the getGazetteerSources will provide you will a list of MarineRegions source information. There is no arguments or query strings to provide to this call.

In [ ]:
# Getting the list of gazetteer sources

# /getGazetteerSources
req = requests.get(f'{marineRegions_url}/getGazetteerSources.{API_EXT}/')
req.json()

### Get Gazetteers from a Specific Source
If you know the name exact source name you can use that value in the call to *getGazetteerRecordsBySource* to return only the gazetteer records from that source.

In [ ]:
# Picking only the gazetteer records for SAIL
source = "SAIL"

#/getGazetteerRecordsBySource
req = requests.get(f'{marineRegions_url}/getGazetteerRecordsBySource.{API_EXT}/{source}/')
sail_data = req.json()

In [ ]:
# For every point that has a location lets map those points to the map 
sail_map = folium.Map()
for point in sail_data:
    try:
        p = folium.Marker([point['latitude'], point['longitude']], popup=point)
        sail_map.add_child(p)
    except:
        pass
    
sail_map

## Get Gazetteer by MGRID
A MarineRegions MGRID is used to query for a specific Gazetteer record.

## Gazetteer Records
Returns the gazetteer records for a given MGRID

In [ ]:
mgrid = "3293" # MGRID for Belgian Exclusive Economic Zone

req = requests.get(f'{marineRegions_url}/getGazetteerRecordByMRGID.{API_EXT}/{mgrid}/')
req.json()

## Gazetteer Names
Just returns the name values for a MGRID

In [ ]:
mgrid = "3293"

# /getGazetteerNamesByMRGID
response = requests.get(f'{marineRegions_url}/getGazetteerNamesByMRGID.{API_EXT}/{mgrid}/')
response_json = response.json()
print_json(response_json)

## Get Gazetteer Relations by MGRID
Returns related Gazetteer records given a valid MGRID.

Relations are defined by the type variable.

In [ ]:
# What are all the marine regions related to Belgian Exclusive Economic Zone?
mgrid = "3293"
direction = 'both' #Must be upper, lower, or both
rtype = 'all' #Must be either partof,partlypartof,adjecentto,similarto,administrativepartof,influencedby or all

req = requests.get(f'{marineRegions_url}/getGazetteerRelationsByMRGID.{API_EXT}/{mgrid}/{direction}/{rtype}')
related_belgian_eco_zone = response.json()

related_belgian_eco_zone

In [ ]:
# Lets map all the related gazetteer records
m = folium.Map(location=[51.46545, 2.70504],zoom_start=8)
for point in related_belgian_eco_zone:
    try:
        if point['MRGID'] == 3293:
            p = folium.Marker([point['latitude'], point['longitude']], popup=point, icon=folium.Icon(color='red'))
            m.add_child(p)
        else:
            p = folium.Marker([point['latitude'], point['longitude']], popup=point)
            m.add_child(p)
    except:
        pass
    
m

## Get Gazetteer WMSes
The WMS records contain urls to layers that can be used in GIS packages such as ArcGIS and QGIS.

In [ ]:
mgrid = "3293"

req = requests.get(f'{marineRegions_url}/getGazetteerWMSes.{API_EXT}/{mgrid}/')
req.json()

## Get Gazetteer Records by Latitude/Longitude
Return Gazetteer records within the default radius of a specific lat/lon

In [ ]:
lat = 45.0 #Double, from 0 to 90
lon = 0 #Double, from -180 to 180
#Default radius is 5 degrees for both lat and lon

req = requests.get(f'{marineRegions_url}/getGazetteerRecordsByLatLong.{API_EXT}/{lat}/{lon}/')
map_points = req.json()
map_points

In [ ]:
# Map the results
byradius_map = folium.Map(location=[lat, lon],zoom_start=5)
for point in map_points:
    try:
            p = folium.Marker([point['latitude'], point['longitude']], popup=point)
            byradius_map.add_child(p)
    except:
        pass
    
byradius_map

## Get Gazetteer Types
Retrieve all the Gazetteer Types along with their descriptions.

In [ ]:
#/getGazetteerTypes
req = requests.get(f'{marineRegions_url}/getGazetteerTypes.{API_EXT}/')
print_json(req.json())